In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from pysat.formula import CNF
import networkx as nx
import numpy as np
import pickle

import os

def _build_graph(cnf_file, output_file):
    cnf = CNF(cnf_file)
    nv = cnf.nv
    clauses = list(filter(lambda x: x, cnf.clauses))
    ind = { k:[] for k in np.concatenate([np.arange(1, nv+1), -np.arange(1, nv+1)]) }
    edges = []
    for i, clause in enumerate(clauses):
        a = clause[0]
        b = clause[1]
        c = clause[2]
        aa = 3 * i + 0
        bb = 3 * i + 1
        cc = 3 * i + 2
        ind[a].append(aa)
        ind[b].append(bb)
        ind[c].append(cc)
        edges.append((aa, bb))
        edges.append((aa, cc))
        edges.append((bb, cc))

    for i in np.arange(1, nv+1):
        for u in ind[i]:
            for v in ind[-i]:
                edges.append((u, v))

    G = nx.from_edgelist(edges)

    # write graph object to output file
    with open(output_file, "wb") as f:
        pickle.dump(G, f, pickle.HIGHEST_PROTOCOL)

# gut = []

# with open('./satlib_graphs.txt') as f:
#     gut = f.read().splitlines() 

# print(gut)

# print("hi")

# graph_directory = "./SATLIB/"

# datasets = []
# i = 0
# for filename in os.listdir(graph_directory):
#     if filename.endswith(".cnf"):
#         if f"{filename[:-4]}.gpickle" in gut:
#             i = i+1
#             print(i)
#             _build_graph(f"./SATLIB/{filename}", f"./output/{filename[:-4]}.gpickle")

## Import Graphs

In [3]:
import os
import networkx as nx
import pickle

graph_directory = "./output"

datasets = []

for filename in os.listdir(graph_directory):
    if filename.endswith(".gpickle"):
        print("3SAT Graph ", os.path.join(graph_directory, filename), "is being imported ...")
        with open(os.path.join(graph_directory, filename), 'rb') as f:
            G = pickle.load(f)
            datasets.append(
            {
                "name": filename[:-8],
                "graph": nx.relabel.convert_node_labels_to_integers(
                    G, first_label=0
                ),
            })
datasets = datasets[0:5] + datasets[7:8] + datasets[10:13] + datasets[14:16] + datasets[17:18] + datasets[37:38] + datasets[39:40] + datasets[19:20] + datasets[21:22]
print(len(datasets))

3SAT Graph  ./output/CBS_k3_n100_m418_b50_780.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m441_b70_445.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m441_b30_978.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m418_b10_148.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m449_b10_868.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m418_b30_414.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m441_b50_480.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m403_b30_699.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m441_b50_737.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m449_b10_931.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m403_b50_163.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m411_b30_323.gpickle is being imported ...
3SAT Graph  ./output/CBS_k3_n100_m411_b10_687.gpickle is being imported ...
3SAT Graph  

In [4]:
from solvers.dNNMIS_GPU_TAU import DNNMIS as V2
from solvers.KaMIS import ReduMIS
from solvers.ILPMIS import ILPMIS
from solvers.Quadratic import Quadratic

solvers = [
    # {
    #     "name": "dNN V2 GPU",
    #     "class": V2,
    #     "params": {
    #         "learning_rate": 0.05,
    #         "selection_criteria": 0.45,
    #         "max_steps": 20000,
    #         "use_cpu": True
    #     },
    # },
    {
        "name": "Quadratic GPU",
        "class": Quadratic,
        "params": {
            "learning_rate": 0.3,
            "max_steps": 75000
        },
    },
    # {
    #     "name": "dNN",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.8,
    #         "max_steps": 25000,
    #     },
    # },
    #     {
    #     "name": "dNN w/SG5k",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.65,
    #         "max_steps": 80000,
    #         "max_subgraph_steps": 5000,
    #     },
    # },
    # {"name": "ReduMIS", "class": ReduMIS, "params": {"seed": 13}},
    # {"name": "ILP", "class": ILPMIS, "params": {"time_limit": 935}}
]

In [5]:
from copy import deepcopy

def benchmark(datasets, solvers):
    solutions = []

    stage = 0
    stages = len(solvers) * len(datasets)

    for solver in solvers:
        for dataset in datasets:
            solver_instance = solver["class"](dataset["graph"], solver["params"])
            solver_instance.solve()
            solution = {
                "solution_method": solver["name"],
                "dataset_name": dataset["name"],
                "data": deepcopy(solver_instance.solution),
                "time_taken": deepcopy(solver_instance.solution_time),
            }
            solutions.append(solution)
            del solver_instance
            stage += 1
            print(f"Completed {stage} / {stages}")

    return solutions

solutions = benchmark(datasets, solvers)

/home/cledenmat/github.com/ledenmat/DatalessNN/solvers/Quadratic.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adjacency_matrix_tensor = torch.tensor(adjacency_matrix_dense, dtype=torch.float32)
/home/cledenmat/github.com/ledenmat/DatalessNN/solvers/Quadratic.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adjacency_matrix_tensor_comp = torch.tensor(adjacency_matrix_dense_comp, dtype=torch.float32)


using device:  cuda:0


/home/cledenmat/github.com/ledenmat/DatalessNN/solvers/Quadratic.py:18: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp) @ Matrix_X)


Step 1000/75000, IS: [1, 4, 8, 13, 16, 18, 22, 24, 28, 30, 33, 36, 40, 44, 47, 50, 52, 56, 58, 61, 65, 67, 69, 72, 75, 79, 83, 86, 87, 90, 93, 96, 100, 103, 105, 108, 112, 116, 118, 121, 125, 126, 131, 134, 135, 138, 141, 144, 148, 150, 154, 157, 159, 163, 165, 168, 172, 176, 177, 180, 185, 186, 191, 192, 195, 200, 202, 204, 207, 211, 213, 216, 219, 224, 227, 229, 233, 234, 238, 242, 245, 247, 251, 254, 256, 258, 261, 267, 270, 273, 276, 280, 282, 285, 289, 292, 294, 297, 301, 304, 306, 309, 314, 317, 318, 322, 325, 327, 330, 334, 338, 342, 345, 350, 352, 356, 358, 360, 364, 366, 369, 373, 377, 379, 383, 385, 389, 391, 395, 398, 399, 402, 405, 410, 411, 416, 417, 420, 423, 430, 432, 437, 438, 443, 445, 447, 451, 455, 457, 460, 462, 465, 469, 471, 474, 478, 480, 484, 486, 491, 492, 495, 503, 507, 511, 515, 516, 521, 523, 525, 529, 531, 535, 539, 541, 543, 546, 550, 552, 556, 560, 562, 565, 570, 575, 576, 583, 585, 590, 593, 596, 598, 601, 605, 607, 609, 613, 615, 618, 622, 625, 629, 630

In [ ]:
import pandas
import matplotlib.pyplot as plt

dataset_index = {k: v for v, k in enumerate([dataset["name"] for dataset in datasets])}
datasets_solutions = [[] for i in range(len(datasets))]

table_data = []

for solution in solutions:
    dsi = dataset_index[solution["dataset_name"]]
    datasets_solutions[dsi].append(solution)

i = 0
for dataset_solutions in datasets_solutions:
    # IS CHECK
    is_check = []
    for solution in dataset_solutions:
        IS_set = solution["data"]["graph_mask"]
        subgraph = datasets[dataset_index[solution["dataset_name"]]]["graph"].subgraph(
            IS_set
        )
        # if len(subgraph.edges) > 0:
        #     plt.figure(i)
        #     plt.title(subgraph.edges)
        #     i += 1
        #     nx.draw(datasets[dataset_index[solution["dataset_name"]]]["graph"], with_labels=True, node_color=IS_set)
        #     is_check.append(False)
        #     print(
        #         f"Non IS found using {solution['solution_method']} on {solution['dataset_name']}"
        #     )
        # else:
        is_check.append(True)

    table_row = [dataset_solutions[0]['dataset_name']]

    table_row.extend([solution["data"]["size"] for solution in dataset_solutions])
    table_row.extend([solution["time_taken"] for solution in dataset_solutions])
    table_row.extend(is_check)

    table_data.append(table_row)

table_headers = ["Dataset Name"]

table_headers.extend([solver["name"] + " Solution Size" for solver in solvers])
table_headers.extend([solver["name"] + " Solution Time" for solver in solvers])
table_headers.extend([solver["name"] + " Solution IS" for solver in solvers])

table = pandas.DataFrame(table_data, columns=table_headers)
table
table.to_csv("results.csv")